In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [1]:
from matplotlib import rcParams, pyplot as plt
import numpy as np
import string
import os
import pandas as pd
from pathlib import Path
import re
from sklearn.metrics import accuracy_score, log_loss
from sklearn.model_selection import StratifiedKFold
import tensorflow as tf
from keras.layers import Input, Dense
from keras.models import Model
from tensorflow.keras import Sequential
from nltk.corpus import stopwords
from tensorflow.keras.layers import Dense, Embedding, LSTM,GRU, GlobalMaxPooling1D, Conv1D, Dropout, Bidirectional, concatenate
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.utils import plot_model, to_categorical
from tensorflow.keras.optimizers import Adam
import warnings 
warnings.filterwarnings(action='ignore')

In [2]:
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    # Restrict TensorFlow to only use the first GPU
    try:
        tf.config.experimental.set_visible_devices(gpus[0], 'GPU')
        logical_gpus = tf.config.experimental.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPU")
    except RuntimeError as e:
        # Visible devices must be set before GPUs have been initialized
        print(e)
else:
    print('No GPU detected')


1 Physical GPUs, 1 Logical GPU


In [3]:
rcParams['figure.figsize'] = (16, 8)
plt.style.use('fivethirtyeight')
pd.set_option('max_columns', 100)
pd.set_option("display.precision", 4)
warnings.simplefilter('ignore')

In [4]:
data_dir = Path('/content/drive/MyDrive/dacon/input')
feature_dir = Path('../build/feature')
val_dir = Path('/content/drive/MyDrive/dacon/build/val')
tst_dir = Path('/content/drive/MyDrive/dacon/build/tst')
sub_dir = Path('/content/drive/MyDrive/dacon/build/sub')

trn_file = data_dir / 'train.csv'
tst_file = data_dir / 'test_x.csv'
sample_file = data_dir / 'sample_submission.csv'

target_col = 'author'
n_fold = 5
n_class = 5
seed = 42

In [5]:
algo_name = 'lstm'
feature_name = 'emb'
model_name = f'{algo_name}_{feature_name}'

feature_file = feature_dir / f'{feature_name}.csv'
p_val_file = val_dir / f'{model_name}.val.csv'
p_tst_file = tst_dir / f'{model_name}.tst.csv'
sub_file = sub_dir / f'{model_name}.csv'

In [11]:
train = pd.read_csv(trn_file, index_col=0)
train.head()

,text,author
index,,
0,"He was almost choking. There was so much, so m...",3
1,"“Your sister asked for it, I suppose?”",2
2,"She was engaged one day as she walked, in per...",1
3,"The captain was in the porch, keeping himself ...",4
4,"“Have mercy, gentlemen!” odin flung up his han...",3


In [12]:
test = pd.read_csv(tst_file, index_col=0)
test.head()

,text
index,
0,“Not at all. I think she is one of the most ch...
1,"""No,"" replied he, with sudden consciousness, ""..."
2,As the lady had stated her intention of scream...
3,“And then suddenly in the silence I heard a so...
4,His conviction remained unchanged. So far as I...


In [9]:
train[train['text'].str.contains("makin'")]

,text,author
index,,
47126,"'We fand her in odin Bay, Rorie an' me, and a'...",4


In [10]:
from tensorflow.keras.preprocessing.text import text_to_word_sequence
puncts = [',', '.', '"', ':', ')', '(', '-', '!', '?', '|', ';', "'", '$', '&', '/', '[', ']',
          '>', '%', '=', '#', '*', '+', '\\', '•', '~', '@', '£', '·', '_', '{', '}', '©', '^',
          '®', '`', '<', '→', '°', '€', '™', '›', '♥', '←', '×', '§', '″', '′', 'Â', '█',
          '½', 'à', '…', '“', '★', '”', '–', '●', 'â', '►', '−', '¢', '²', '¬', '░', '¶',
          '↑', '±', '¿', '▾', '═', '¦', '║', '―', '¥', '▓', '—', '‹', '─', '▒', '：', '¼',
          '⊕', '▼', '▪', '†', '■', '’', '▀', '¨', '▄', '♫', '☆', 'é', '¯', '♦', '¤', '▲',
          'è', '¸', '¾', 'Ã', '⋅', '‘', '∞', '∙', '）', '↓', '、', '│', '（', '»', '，', '♪',
          '╩', '╚', '³', '・', '╦', '╣', '╔', '╗', '▬', '❤', 'ï', 'Ø', '¹', '≤', '‡', '√']
result=set()

def find_puncts(text):
    global result
    words = text_to_word_sequence(text)
    for word in words :
      for character in word :
        if character in puncts : result.add(character)

    return

train['text'].str.lower().apply(find_puncts)
result

{"'", '£', 'à', 'â', 'è', 'é', 'ï', '—', '‘', '’', '“', '”'}

In [11]:
train['text'] = train['text'].str.lower()
test['text'] = test['text'].str.lower()

In [13]:
train['text'] = train['text'].str.replace('\?',' quesmark ')
train['text'] = train['text'].str.replace('\!',' exclmark ')
train['text'] = train['text'].str.replace('\&',' empent ')
train['text'] = train['text'].str.replace("\*",' chstar ')
train['text'] = train['text'].str.replace(";",' smcolons  ')

test['text'] = test['text'].str.replace('\?',' quesmark ')
test['text'] = test['text'].str.replace('\!',' exclmark ')
test['text'] = test['text'].str.replace('\&',' empent ')
test['text'] = test['text'].str.replace("\*",' chstar ')
test['text'] = test['text'].str.replace(";",' smcolons  ')

In [13]:

contraction = set()

def find_contraction(text):
    global contraction
    words = text_to_word_sequence(text)
    for word in words :
      if "\'" in word or "’" in word : contraction.add(word)       
    return 

train['text'].str.lower().apply(find_contraction)
contraction    

{"order'",
 "lucy's",
 "'bah",
 'seaman’s',
 'people’s',
 "'ooman",
 "'at",
 'ye’ll',
 "minutes'",
 "dog's",
 "sweetheart's",
 "'eccentricity",
 "about'",
 "anhalt's",
 "'confidentially'",
 'men’s',
 "'cod",
 'ship’s',
 'for’',
 'you—there’s',
 "bachelor's",
 "'twas",
 'again’—here',
 'odin’d',
 "steward's",
 'lawful’',
 "'lots",
 "matron's",
 "'god",
 'again—that’s',
 'ham’s',
 "poet's",
 "gude's",
 'sentry’s',
 "companion's",
 "qu'avez",
 'hart’',
 'joe’s',
 "“we're",
 "'eh",
 'companion’s',
 "individual's",
 "flunkey's",
 '‘’twas',
 "pushkin's",
 'iou’s',
 "mutineers'",
 '‘that’s',
 "detective's",
 '‘tom’',
 "b'ye",
 "buryin'",
 'doctor’s',
 '‘hosannah’',
 'woodman’s',
 'dog’s',
 "neighbour's",
 '“‘there’s',
 "parson's",
 '—odin’s',
 "d'esprit",
 "'how's",
 'boys’',
 "time's",
 'papa’s',
 "'dot",
 "agin'",
 "'step",
 "s'appelle",
 'dupont’s',
 "'toby",
 'conviction’',
 "“'not",
 'metcalf’s',
 'andrew’s',
 "mornin'",
 "sins'",
 'semionovitch’s',
 "'hah",
 "rollin'",
 "'isn't",
 "let'

In [14]:
train['text']=train['text'].str.replace('\'s', '')
train['text']=train['text'].str.replace('’s', '')
train['text']=train['text'].str.replace('\'ll', '')
train['text']=train['text'].str.replace('’ll', '')
train['text']=train['text'].str.replace("\'", '')
train['text']=train['text'].str.replace("’", '')

test['text']=test['text'].str.replace("’s",'')
test['text']=test['text'].str.replace("\'s",'')
test['text']=test['text'].str.replace('\'ll', '')
test['text']=test['text'].str.replace('’ll', '')
test['text']=test['text'].str.replace("\'", '')
test['text']=test['text'].str.replace("’", '')

In [15]:
train['text']=train['text'].str.replace('á', '')
train['text']=train['text'].str.replace('à', '')
train['text']=train['text'].str.replace('â', '')
train['text']=train['text'].str.replace('ä', '')
train['text']=train['text'].str.replace('é', '')
train['text']=train['text'].str.replace('í', '')
train['text']=train['text'].str.replace('ï', '')
train['text']=train['text'].str.replace('ó', '')
train['text']=train['text'].str.replace('ú', '')
train['text']=train['text'].str.replace('ý', '')
train['text']=train['text'].str.replace('ü', '')
train['text']=train['text'].str.replace('è', '')
train['text']=train['text'].str.replace('£', '')

test['text']=test['text'].str.replace('ä', '')
test['text']=test['text'].str.replace('â', '')
test['text']=test['text'].str.replace('à', '')
test['text']=test['text'].str.replace('á', '')
test['text']=test['text'].str.replace('é', '')
test['text']=test['text'].str.replace('ï', '')
test['text']=test['text'].str.replace('í', '')
test['text']=test['text'].str.replace('ó', '')
test['text']=test['text'].str.replace('ú', '')
test['text']=test['text'].str.replace('ý', '')
test['text']=test['text'].str.replace('ü', '')
test['text']=test['text'].str.replace('è', '')
test['text']=test['text'].str.replace('£', '')

In [16]:
def alpha_num(text):
    return re.sub("\d+", ' num ', text)

train['text']=train['text'].apply(alpha_num)
test['text']=test['text'].apply(alpha_num)

In [17]:
train['text']=train['text'].str.replace('\(', ' \( ')
train['text']=train['text'].str.replace('\{', ' \{ ')
train['text']=train['text'].str.replace('\[', ' \[ ')
train['text']=train['text'].str.replace('\)', ' \) ')
train['text']=train['text'].str.replace('\}', ' \} ')
train['text']=train['text'].str.replace('\]', ' \] ')
train['text']=train['text'].str.replace('—', '')
train['text']=train['text'].str.replace('_', '')
train['text']=train['text'].str.replace(':', '')
train['text']=train['text'].str.replace("‘",' ‘ ')
train['text']=train['text'].str.replace("“",' “ ')

test['text']=test['text'].str.replace('\(', ' \( ')
test['text']=test['text'].str.replace('\{', ' \{ ')
test['text']=test['text'].str.replace('\[', ' \[ ')
test['text']=test['text'].str.replace('\)', ' \) ')
test['text']=test['text'].str.replace('\}', ' \} ')
test['text']=test['text'].str.replace('\]', ' \] ')
test['text']=test['text'].str.replace('—', '')
test['text']=test['text'].str.replace('_', '')
test['text']=test['text'].str.replace(':', '')
test['text']=test['text'].str.replace("‘",' ‘ ')
test['text']=test['text'].str.replace("“",' “ ')

In [18]:
import nltk
nltk.download('wordnet')
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [19]:
from nltk.stem.wordnet import WordNetLemmatizer 
from nltk.tokenize import TweetTokenizer

def lemma_text(text):
    tokenizer=TweetTokenizer()
    words=tokenizer.tokenize(text)
    lem = WordNetLemmatizer()
    words=[lem.lemmatize(word, "v") for word in words]
    
    clean_sent=" ".join(words)
    
    return clean_sent

train['text'] = train['text'].str.lower().apply(lemma_text)
test['text'] = test['text'].str.lower().apply(lemma_text)

In [20]:
X_train = train['text'].values
X_test = test['text'].values
y = train['author'].values
print(X_train.shape, X_test.shape, y.shape)

(54879,) (19617,) (54879,)


In [21]:

vocab_size = 21000
embedding_dim = 128
max_length = 300
padding_type='post'

In [22]:
tokenizer = Tokenizer(num_words = vocab_size)
tokenizer.fit_on_texts(X_train)
word_index = tokenizer.word_index

In [23]:
train_sequences = tokenizer.texts_to_sequences(X_train)
test_sequences = tokenizer.texts_to_sequences(X_test)

In [24]:
trn = pad_sequences(train_sequences, padding=padding_type, maxlen=max_length)
tst = pad_sequences(test_sequences, padding=padding_type, maxlen=max_length)
print(trn.shape, tst.shape)

(54879, 300) (19617, 300)


In [25]:
cv = StratifiedKFold(n_splits=n_fold, shuffle=True, random_state=seed)

In [26]:
def get_model():
    model = Sequential([
        Embedding(vocab_size, embedding_dim, input_length=max_length),
        Bidirectional(LSTM(128, return_sequences=True)),
        Bidirectional(LSTM(128)),
        Dense(n_class, activation='softmax')
    ])
    
    model.compile(loss='categorical_crossentropy', optimizer=Adam(learning_rate=.01))
    return model

In [27]:
p_val = np.zeros((trn.shape[0], n_class))
p_tst = np.zeros((tst.shape[0], n_class))
for i, (i_trn, i_val) in enumerate(cv.split(trn, y), 1):
    print(f'training model for CV #{i}')
    clf = get_model()
    
    es = EarlyStopping(monitor='val_loss', min_delta=0.001, patience=3,
                       verbose=1, mode='min', baseline=None, restore_best_weights=True)

    clf.fit(trn[i_trn], 
            to_categorical(y[i_trn]),
            validation_data=(trn[i_val], to_categorical(y[i_val])),
            epochs=10,
            batch_size=256,
            callbacks=[es])
    p_val[i_val, :] = clf.predict(trn[i_val])
    p_tst += clf.predict(tst) / n_fold

training model for CV #1
Epoch 1/10
172/172 [==============================] - 37s 214ms/step - loss: 1.0604 - val_loss: 0.7176
Epoch 2/10
172/172 [==============================] - 36s 212ms/step - loss: 0.5665 - val_loss: 0.6222
Epoch 3/10
172/172 [==============================] - 36s 209ms/step - loss: 0.4157 - val_loss: 0.5916
Epoch 4/10
172/172 [==============================] - 36s 210ms/step - loss: 0.3253 - val_loss: 0.6413
Epoch 5/10
172/172 [==============================] - 36s 209ms/step - loss: 0.2685 - val_loss: 0.7328
Epoch 6/10
172/172 [==============================] - 36s 210ms/step - loss: 0.2236 - val_loss: 0.7825
Epoch 00006: early stopping
training model for CV #2
Epoch 1/10
172/172 [==============================] - 37s 215ms/step - loss: 1.0257 - val_loss: 0.7104
Epoch 2/10
172/172 [==============================] - 36s 208ms/step - loss: 0.5494 - val_loss: 0.6087
Epoch 3/10
172/172 [==============================] - 36s 208ms/step - loss: 0.3981 - val_loss: 0.

In [28]:
print(f'Accuracy (CV): {accuracy_score(y, np.argmax(p_val, axis=1)) * 100:8.4f}%')
print(f'Log Loss (CV): {log_loss(pd.get_dummies(y), p_val):8.4f}')

Accuracy (CV):  77.9570%
Log Loss (CV):   0.6053


In [29]:
np.savetxt(p_val_file, p_val, fmt='%.6f', delimiter=',')
np.savetxt(p_tst_file, p_tst, fmt='%.6f', delimiter=',')


In [30]:
sub = pd.read_csv(sample_file, index_col=0)
print(sub.shape)
sub.head()

(19617, 5)


,0,1,2,3,4
index,,,,,
0,0,0,0,0,0
1,0,0,0,0,0
2,0,0,0,0,0
3,0,0,0,0,0
4,0,0,0,0,0


In [31]:

sub[sub.columns] = p_tst
sub.head()

,0,1,2,3,4
index,,,,,
0,0.0305,0.4482,0.4279,0.0869,0.0065
1,0.4240,0.2200,0.0225,0.1035,0.2301
2,0.9835,0.0101,0.0014,0.0013,0.0037
3,0.0043,0.0081,0.9763,0.0058,0.0056
4,0.7613,0.0171,0.0305,0.0686,0.1225


In [32]:

sub.to_csv(sub_file)